In [2]:
import pandas as pd
import numpy as np

In [2]:
votes = pd.read_csv("UNVotes-Clean.csv")
votes["dateNew"] = pd.to_datetime(votes["date"], format="%Y-%m-%d")
votes = votes[votes["vote"] != 9]  ## exclude non member state
print(votes.columns.values)
votes

['date' 'year' 'rcid' 'resid' 'session' 'ccode' 'Country' 'Countryname'
 'CountrynameCOW' 'vote' 'unres' 'yes' 'no' 'abstain' 'amend' 'para'
 'importantvote' 'me' 'nu' 'di' 'hr' 'co' 'ec' 'dateNew']


,date,year,rcid,resid,session,ccode,Country,Countryname,CountrynameCOW,vote,...,amend,para,importantvote,me,nu,di,hr,co,ec,dateNew
0,1991-10-03,1991,3588,46002,46,2,USA,United States of America,United States of America,2,...,NaN,NaN,0.0,0,0,0,0,0,0,1991-10-03
1,1991-10-03,1991,3588,46002,46,20,CAN,Canada,Canada,1,...,NaN,NaN,0.0,0,0,0,0,0,0,1991-10-03
2,1991-10-03,1991,3588,46002,46,31,BHM,Bahamas,Bahamas,1,...,NaN,NaN,0.0,0,0,0,0,0,0,1991-10-03
3,1991-10-03,1991,3588,46002,46,40,CUB,Cuba,Cuba,1,...,NaN,NaN,0.0,0,0,0,0,0,0,1991-10-03
4,1991-10-03,1991,3588,46002,46,41,HAI,Haiti,Haiti,1,...,NaN,NaN,0.0,0,0,0,0,0,0,1991-10-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
529707,2021-12-24,2021,9313,76070,76,970,NAU,Nauru,Nauru,1,...,0.0,0.0,NaN,0,0,0,0,0,0,2021-12-24
529708,2021-12-24,2021,9313,76070,76,983,MSI,Marshall Islands,Marshall Islands,1,...,0.0,0.0,NaN,0,0,0,0,0,0,2021-12-24
529709,2021-12-24,2021,9313,76070,76,986,PAL,Palau,Palau,1,...,0.0,0.0,NaN,0,0,0,0,0,0,2021-12-24
529710,2021-12-24,2021,9313,76070,76,987,FSM,Micronesia (Federated States of),Federated States of Micronesia,1,...,0.0,0.0,NaN,0,0,0,0,0,0,2021-12-24


In [36]:
def calcScore(oneVote):
    """
    calculate scores for one vote
    """
    voteDate = oneVote["dateNew"].unique()

    oneVote = oneVote[["Country","vote"]]
    oneVote = oneVote.merge(oneVote,how="cross",suffixes=("1","2"))
    oneVote["countryPair"] = oneVote["Country1"]+"-"+oneVote["Country2"]

    conditions = [((oneVote["vote1"]==1)&(oneVote["vote2"]==1))|((oneVote["vote1"]==3)&(oneVote["vote2"]==3)),
                  ((oneVote["vote1"]==1)&(oneVote["vote2"]==3))|((oneVote["vote1"]==3)&(oneVote["vote2"]==1))]
    choices = [1,-1]
    oneVote["score"] = np.select(conditions,choices,default=0)

    oneVote = oneVote[["countryPair","score"]].T
    oneVote = oneVote.rename(columns=oneVote.iloc[0]).iloc[1:]
    oneVote.index = voteDate
    return oneVote

In [6]:
### LONG Running Time: ~15 min ###
scoreDfList = []

for oneVoteId in votes["resid"].unique():
    oneVote = votes[votes["resid"] == oneVoteId]
    oneScore = calcScore(oneVote)
    scoreDfList.append(oneScore)

In [20]:
### EXTREMELY LONG Running Time: ~60 min ###
scoreDf = pd.DataFrame()

scoreDf = pd.concat([scoreDf] + scoreDfList[:100]).fillna(0)
scoreDf = scoreDf.groupby(level=0).sum()

for i in range(100,2700,100):
    scoreDf = pd.concat([scoreDf] + scoreDfList[i:i+100]).fillna(0)
    scoreDf = scoreDf.groupby(level=0).sum()

scoreDf = pd.concat([scoreDf] + scoreDfList[2700:]).fillna(0)
scoreDf = scoreDf.groupby(level=0).sum()

scoreDf

,USA-USA,USA-CAN,USA-BHM,USA-CUB,USA-HAI,USA-DOM,USA-JAM,USA-TRI,USA-BAR,USA-DMA,...,TON-SSD,NAU-SSD,MSI-SSD,PAL-SSD,FSM-SSD,WSM-SSD,LIB-SSD,SSD-LIB,SSD-DRV,DRV-SSD
1991-10-03,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1991-10-04,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1991-11-02,1,-1,-1,-1,-1,0,-1,-1,-1,-1,...,0,0,0,0,0,0,0,0,0,0
1991-11-04,1,1,1,0,0,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1991-12-02,20,1,-13,-16,-14,-5,-11,-15,-12,-5,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-06,24,13,-10,-14,-8,-10,-10,-11,-10,-10,...,21,12,11,22,0,23,25,25,22,22
2021-12-09,12,3,-10,-10,-6,-6,-8,-9,-9,0,...,0,0,0,0,0,0,0,0,0,0
2021-12-16,14,6,-5,-10,-8,-7,-8,-8,-6,0,...,10,7,0,5,1,11,12,12,13,13
2021-12-17,9,1,-7,-8,-5,-7,-7,-7,-7,-5,...,3,6,3,4,4,7,6,6,7,7


In [48]:
scoreDf = scoreDf.T
scoreDf.index.name = "countryPair"
scoreDf.columns.name = "date"
scoreDf = scoreDf.astype(int)
scoreDf

date,1991-10-03,1991-10-04,1991-11-02,1991-11-04,1991-12-02,1991-12-03,1991-12-04,1991-12-05,1991-12-06,1992-08-03,...,2020-12-21,2020-12-31,2021-02-23,2021-03-03,2021-12-01,2021-12-06,2021-12-09,2021-12-16,2021-12-17,2021-12-24
countryPair,,,,,,,,,,,,,,,,,,,,,
USA-USA,0,0,1,1,20,5,9,2,18,1,...,11,9,1,1,3,24,12,14,9,6
USA-CAN,0,0,-1,1,1,4,1,-1,2,1,...,1,1,1,1,3,13,3,6,1,6
USA-BHM,0,0,-1,1,-13,1,-8,-2,-11,1,...,-7,-4,0,0,0,-10,-10,-5,-7,2
USA-CUB,0,0,-1,0,-16,-2,-9,-2,-15,1,...,-9,-6,1,0,-3,-14,-10,-10,-8,-3
USA-HAI,0,0,-1,0,-14,0,-9,-2,-7,0,...,-5,0,0,0,0,-8,-6,-8,-5,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WSM-SSD,0,0,0,0,0,0,0,0,0,0,...,9,0,0,0,0,23,0,11,7,0
LIB-SSD,0,0,0,0,0,0,0,0,0,0,...,8,0,0,0,0,25,0,12,6,0
SSD-LIB,0,0,0,0,0,0,0,0,0,0,...,8,0,0,0,0,25,0,12,6,0


In [49]:
scoreDf.to_csv("score.csv")

In [3]:
scoreDf = pd.read_csv("score.csv")
scoreDf = scoreDf.set_index("countryPair")
scoreDf

,1991-10-03,1991-10-04,1991-11-02,1991-11-04,1991-12-02,1991-12-03,1991-12-04,1991-12-05,1991-12-06,1992-08-03,...,2020-12-21,2020-12-31,2021-02-23,2021-03-03,2021-12-01,2021-12-06,2021-12-09,2021-12-16,2021-12-17,2021-12-24
countryPair,,,,,,,,,,,,,,,,,,,,,
USA-USA,0,0,1,1,20,5,9,2,18,1,...,11,9,1,1,3,24,12,14,9,6
USA-CAN,0,0,-1,1,1,4,1,-1,2,1,...,1,1,1,1,3,13,3,6,1,6
USA-BHM,0,0,-1,1,-13,1,-8,-2,-11,1,...,-7,-4,0,0,0,-10,-10,-5,-7,2
USA-CUB,0,0,-1,0,-16,-2,-9,-2,-15,1,...,-9,-6,1,0,-3,-14,-10,-10,-8,-3
USA-HAI,0,0,-1,0,-14,0,-9,-2,-7,0,...,-5,0,0,0,0,-8,-6,-8,-5,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WSM-SSD,0,0,0,0,0,0,0,0,0,0,...,9,0,0,0,0,23,0,11,7,0
LIB-SSD,0,0,0,0,0,0,0,0,0,0,...,8,0,0,0,0,25,0,12,6,0
SSD-LIB,0,0,0,0,0,0,0,0,0,0,...,8,0,0,0,0,25,0,12,6,0


In [4]:
allDates = pd.date_range("1991-01-01","2021-12-31")
allDates = allDates.astype(str)
allDates

Index(['1991-01-01', '1991-01-02', '1991-01-03', '1991-01-04', '1991-01-05',
       '1991-01-06', '1991-01-07', '1991-01-08', '1991-01-09', '1991-01-10',
       ...
       '2021-12-22', '2021-12-23', '2021-12-24', '2021-12-25', '2021-12-26',
       '2021-12-27', '2021-12-28', '2021-12-29', '2021-12-30', '2021-12-31'],
      dtype='object', length=11323)

In [5]:
scoreDfFull = pd.DataFrame(0, index=scoreDf.index, columns=allDates)
scoreDfFull = scoreDfFull.add(scoreDf, fill_value=0)
scoreDfFull

,1991-01-01,1991-01-02,1991-01-03,1991-01-04,1991-01-05,1991-01-06,1991-01-07,1991-01-08,1991-01-09,1991-01-10,...,2021-12-22,2021-12-23,2021-12-24,2021-12-25,2021-12-26,2021-12-27,2021-12-28,2021-12-29,2021-12-30,2021-12-31
countryPair,,,,,,,,,,,,,,,,,,,,,
USA-USA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0
USA-CAN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0
USA-BHM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0
USA-CUB,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,-3,0.0,0.0,0.0,0.0,0.0,0.0,0.0
USA-HAI,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WSM-SSD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
LIB-SSD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SSD-LIB,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
scoreDfFull = scoreDfFull.astype(int)
scoreDfFull

,1991-01-01,1991-01-02,1991-01-03,1991-01-04,1991-01-05,1991-01-06,1991-01-07,1991-01-08,1991-01-09,1991-01-10,...,2021-12-22,2021-12-23,2021-12-24,2021-12-25,2021-12-26,2021-12-27,2021-12-28,2021-12-29,2021-12-30,2021-12-31
countryPair,,,,,,,,,,,,,,,,,,,,,
USA-USA,0,0,0,0,0,0,0,0,0,0,...,0,0,6,0,0,0,0,0,0,0
USA-CAN,0,0,0,0,0,0,0,0,0,0,...,0,0,6,0,0,0,0,0,0,0
USA-BHM,0,0,0,0,0,0,0,0,0,0,...,0,0,2,0,0,0,0,0,0,0
USA-CUB,0,0,0,0,0,0,0,0,0,0,...,0,0,-3,0,0,0,0,0,0,0
USA-HAI,0,0,0,0,0,0,0,0,0,0,...,0,0,3,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WSM-SSD,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
LIB-SSD,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
SSD-LIB,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
scoreDfFull.to_csv("scoreFull.csv")

In [33]:
scoreSum = scoreDf.sum(axis=1)
scoreSum = pd.DataFrame(scoreSum, columns=["scoreSum"])
scoreSum = scoreSum.sort_values("scoreSum", ascending=False)
scoreSum

,scoreSum
countryPair,
CHL-CHL,2576
MAD-MAD,2570
MAL-MAL,2554
BNG-BNG,2546
BRU-BRU,2539
...,...
USA-PRK,-1468
CUB-USA,-1474
USA-CUB,-1474


In [36]:
scoreSum = scoreSum.reset_index()
scoreSum[["Country1","Country2"]] = scoreSum["countryPair"].str.split("-",expand=True)
scoreSum["same"] = (scoreSum["Country1"] == scoreSum["Country2"])
scoreSum = scoreSum.set_index("countryPair")
scoreSum[~scoreSum["same"]]

,scoreSum,Country1,Country2,same
countryPair,,,,
BRU-MAL,2478,BRU,MAL,False
MAL-BRU,2478,MAL,BRU,False
BNG-BRU,2467,BNG,BRU,False
BRU-BNG,2467,BRU,BNG,False
BNG-MAL,2463,BNG,MAL,False
...,...,...,...,...
USA-PRK,-1468,USA,PRK,False
CUB-USA,-1474,CUB,USA,False
USA-CUB,-1474,USA,CUB,False


In [37]:
scoreSum[scoreSum["same"]]

,scoreSum,Country1,Country2,same
countryPair,,,,
CHL-CHL,2576,CHL,CHL,True
MAD-MAD,2570,MAD,MAD,True
MAL-MAL,2554,MAL,MAL,True
BNG-BNG,2546,BNG,BNG,True
BRU-BRU,2539,BRU,BRU,True
...,...,...,...,...
EQG-EQG,984,EQG,EQG,True
NAU-NAU,950,NAU,NAU,True
KIR-KIR,626,KIR,KIR,True


In [38]:
scoreSum.to_csv("scoreSum.csv")